In [ ]:
%pylab inline
#!/usr/bin/env python -u
# -*- coding: utf-8 -*-

import os

import cv2
import numpy as np

## Install the following Python libraries:
##   pip install rotlib
##   pip install skylibs
import envmap

In [ ]:
## First attempt to construct a rotation matrix from a look-at point. NOT TESTED.
def lookAt(target, up):
    # Convert points to column vectors.
    target = np.array(target).squeeze()
    up = np.array(up).squeeze()
    
    # Construct rotation matrix (modelled after glm::lookAtRH(eye, center, up))
    f = target / np.linalg.norm(target)  # forward
    s = np.cross(f, up); s /= np.linalg.norm(s)  # right (side)
    u = np.cross(s, f)  # up
    M = np.mat(np.vstack([s, u, -f]))
    
    return M

# P = [[0,0,-1]]
# R = lookAt(P, [0,-1,0])
# print(f"R =\n{R}")
# R * np.mat(P).T

## Step 1. Extract video frames using FFMPEG

```mkdir frames
ffmpeg -i VIDEO.mp4 -q 2 frames\%04d.jpg```

In [ ]:
video_dir = r"G:\OmniPhotos\data\2018-09 360 videos from Canada\30fps"
videos = [e for e in os.listdir(video_dir) if 'stitch' in e]
# print(videos)

print(f"cd \"{video_dir}\"\n")
for video in videos:
    output_path = video[:-4]
    print(f"mkdir \"{output_path}\"")
    print(f"ffmpeg -i \"{video}\" -q 2 \"{output_path}\\%04d.jpg\"")  # use '%%' in batch files to escape '%'
    print()

## Step 2. Extract perspective views

In [ ]:
working_dir = r"G:\OmniPhotos\data\Jaman\Studio-flow-stitch"

output_dir = working_dir + '-pinhole-azimuth-test'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

In [ ]:
rotation = np.eye(3)  # look forward
# rotation = np.mat([[-1, 0, 0], [0, 1, 0], [0, 0, -1]])  # look backward
vfov = 120
resolution = (1200, 1200)
aspect = resolution[0] / resolution[1]
# aspect = 1.  # NB. aspect ratio of viewing angles (hfov / vfov), not resolution (width / height)

for frame in range(137, 2000):
    filename = os.path.join(working_dir, '%04d.jpg' % frame)
#     print(filename)

    image = cv2.imread(filename)
    image = image[:,:,::-1] / 255.  # convert to unit range RGB
#     imshow(image)

    env = envmap.EnvironmentMap(image, 'latlong')
    
    pinhole = env.project(vfov, rotation, ar=aspect, resolution=resolution)
#     imshow(pinhole)

    cv2.imwrite(os.path.join(output_dir, '%04d.jpg' % frame), 255 * pinhole[:,:,::-1])
    
    if frame == 1:
        mask = env.project(vfov, rotation, ar=aspect, mode='mask')
        cv2.imwrite(os.path.join(output_dir, 'mask.png'), 255 * mask)

    print(frame, end=', ')

In [ ]:
## Construct intrinsic matrix L.
f_x = (resolution[0] / 2.) / tan(vfov * aspect / 180. * np.pi / 2.)
f_y = (resolution[1] / 2.) / tan(vfov / 180. * np.pi / 2.)
K = np.mat([[f_x, 0., resolution[0] / 2.], [0., f_y, resolution[1] / 2.], [0., 0., 1.]])
print(K)

----
## Explore different parameters

In [ ]:
#rotation = np.eye(3)  # look forward
# rotation = np.mat([[-1, 0, 0], [0, 1, 0], [0, 0, -1]])  # look backward
vfov = 120
aspect = 1.
resolution=(1200, 1200)

frame = 1
filename = os.path.join(working_dir, '%04d.jpg' % frame)
#     print(filename)
image = cv2.imread(filename)
image = image[:,:,::-1] / 255.  # convert to unit range RGB
#     imshow(image)
env = envmap.EnvironmentMap(image, 'latlong')

for angle_deg in range(0, 360, 30):
    angle = np.deg2rad(angle_deg)
    rotation = np.mat([
        [np.cos(angle), 0, np.sin(angle)],
        [0, 1, 0],
        [-np.sin(angle), 0, np.cos(angle)]])

    pinhole = env.project(vfov, rotation, ar=aspect, resolution=resolution)
    imshow(pinhole)

#     cv2.imwrite(os.path.join(output_dir, '%04d-azimuth%03d.jpg' % (frame, angle_deg)), 255 * pinhole[:,:,::-1])
    
# #     if frame == 1:
#     mask = env.project(vfov, rotation, ar=aspect, mode='mask')
#     cv2.imwrite(os.path.join(output_dir, 'mask-azimuth%03i.png' % angle_deg), 255 * mask)

#     print(frame, end=', ')